In [27]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import sympy as sp

In [35]:
def funcao(x, y):
    return 4*x[1]**2 + 2*x[0]


def newton_raphson(func, x0, tol=1e-4, max_iter=100):
    x = float(x0)
    xsym = sp.symbols('x')
    fsym = func(xsym)
    dfsy = sp.diff(fsym, xsym)
    df2sy = sp.diff(dfsy, xsym)

    fnum = sp.lambdify(xsym, fsym)
    dfnum = sp.lambdify(xsym, dfsy)
    df2num = sp.lambdify(xsym, df2sy)

    for i in range(max_iter):
        df = dfnum(x)
        df2 = df2num(x)
        x1 = x - dfnum(x) / df2num(x)
        if abs(x1 - x) < tol:
            break
        x = x1
    return x


In [36]:
def gram_schmidt(direcoes, z):
    n = len(z)
    Q = np.zeros((n, n))
    P = np.zeros((n, n))

    Q[:, 0] = z
    for i in range(1, n):
        Q[:, i] = direcoes[:, i - 1]

    if np.linalg.norm(Q[:, 0]) > 0:
        P[:, 0] = Q[:, 0] / np.linalg.norm(Q[:, 0])
    else:
        P[:, 0] = Q[:, 0]

    for i in range(1, n):
        vetor_temp = Q[:, i]
        for j in range(i):
            projecao = np.dot(Q[:, i], P[:, j]) * P[:, j]
            vetor_temp -= projecao

        if np.linalg.norm(vetor_temp) > 0:
            P[:, i] = vetor_temp / np.linalg.norm(vetor_temp)
        else:
            P[:, i] = vetor_temp

    return P

In [37]:
def metodo_rosenbrock(f, x0, s_inicial=0.1, alpha=2.0, beta=0.5, max_iter=10000, tol=1e-6):
    n = len(x0)
    x = np.array(x0, dtype=float)
    passos = np.full(n, s_inicial)
    direcoes = np.identity(n)
    f_x = f(x)

    for _ in range(max_iter):
        x_anterior = x.copy()
        f_anterior = f_x

        for i in range(n):
            x_teste = x + passos[i] * direcoes[:, i]
            f_teste = f(x_teste)

            if f_teste < f_x:
                x = x_teste
                f_x = f_teste
                passos[i] *= alpha
            else:
                passos[i] *= -beta

        if np.linalg.norm(x - x_anterior) < tol:
            break

        if f_x < f_anterior:
            z = x - x_anterior
            direcoes = gram_schmidt(direcoes, z)

    return x, f_x


In [40]:

pontoinicial = np.array([input( "Digite a coordenada x0: "), input( "Digite a coordenada y0: ")], dtype=float)
xmin, fmin = metodo_rosenbrock(funcao, pontoinicial)

plt.figure(figsize=(10, 6))
x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = funcao(X, Y)
contour = plt.contour(X, Y, Z, levels=50, cmap='viridis')
plt.clabel(contour, inline=True, fontsize=8)
plt.scatter(xmin[0], xmin[1], color='red', label='Mínimo encontrado')
plt.legend()
plt.title('Contorno da função de Rosenbrock')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

TypeError: funcao() missing 1 required positional argument: 'y'